In [0]:
#!pip install wikipedia

import pandas as pd
import re
import wikipedia
from bs4 import BeautifulSoup
from lxml import html
import requests
import urllib.request
import threading
import concurrent.futures

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
% cd '/content/drive/My Drive/Colab Notebooks/Omdena'

/content/drive/My Drive/Colab Notebooks/Omdena


In [0]:
dataset = pd.read_csv('df_wiki_try.csv')
small_ds = dataset.head(20).copy()

In [5]:
small_ds

,Unnamed: 0,Index,NAME,ISO,Start year,Num affected,Num homeless,Num injured,wikipedia link
0,0,0,DELLA,JPN,1949,194046.0,NaN,NaN,NaN
1,1,1,RENA,PHL,1949,NaN,NaN,NaN,https://en.wikipedia.org/wiki/Typhoon_Rena_(1949)
2,2,2,JANE,JPN,1950,642117.0,NaN,NaN,https://en.wikipedia.org/wiki/Hurricane_Jane
3,3,3,IRIS,PHL,1951,NaN,NaN,NaN,https://en.wikipedia.org/wiki/Typhoon_Iris_(1951)
4,4,4,CHARLIE,JAM,1951,20000.0,NaN,200.0,https://en.wikipedia.org/wiki/Hurricane_Charli...
5,5,5,RUTH,JPN,1951,NaN,NaN,NaN,https://en.wikipedia.org/wiki/Typhoon_Ruth_(1951)
6,6,6,WANDA,PHL,1951,NaN,NaN,NaN,https://en.wikipedia.org/wiki/Typhoon_Wanda_(1...
7,7,7,AMY,PHL,1951,60000.0,NaN,NaN,https://en.wikipedia.org/wiki/Typhoon_Amy_(1951)
8,8,8,EMMA,PHL,1952,NaN,NaN,103.0,https://en.wikipedia.org/wiki/Typhoon_Emma_(1952)
9,9,9,TRIX,PHL,1952,NaN,NaN,349.0,https://en.wikipedia.org/wiki/Typhoon_Trix_(1952)


In [0]:
global temp_dataset
temp_dataset = small_ds.copy()

In [0]:
# just Enter -> accept default value (this one at the left on input field)
# give yout own digit -> this digit
# any non-digital sign -> do nothig, leave value that is already in dataframe

def acceptResult(data, what):
  data[1] = data[1].replace(',', '').replace('.', '')
  print(data[0])
  print(data[1])
  print(what)
  try:
    temp_dataset.at[int(data[0]), what] = float(data[1])
  except:
    pass
  result_available.set()

In [0]:
regexLetters = u'[A-Za-z]+'

#### regexKetWord, keyWord, targetColumnName -> for editing
regexKeyWord = u'^homeless.*'
keyWord = '.*homeless.*'
targetColumnName = 'Num homeless'

result_available = threading.Event()


def doDefault(sentence):
  global lastDigit
  distance = 0
  sentence = sentence.replace(u'\xa0', u' ')
  words = sentence.split(" ")
  for word in words:
    if len(word)>0:
      distance += 1
    if not bool(re.search(regexLetters, word)):
      lastDigit = word
      distance = 0
    if bool(re.search(regexKeyWord, word)):
      print(str(lastDigit) + " " + str(distance))
      result_available.set()
      return


In [132]:

res = []

def runFor(small_ds):
  for index, row in small_ds.iterrows():
    print()
    print("************************")
    print('row[NAME]  row[Start year]  row[ISO]  row[' + targetColumnName + ']')
    print(row['NAME'] + " " + str(row['Start year']) + " " + row['ISO'] + " " + str(row[targetColumnName]))
    print()

    try:
      url = str(row['wikipedia link'])
      path = '/html/head/title'

      page = urllib.request.urlopen(url, timeout=5)
      soup = BeautifulSoup(page, "lxml")
      response = requests.get(url, timeout=5)
      byte_data = response.content
      source_code = html.fromstring(byte_data)
      title = source_code.xpath(path)
      titleString = title[0].text_content()
      print("TITLE: " + titleString)

      if soup.find(text=re.compile(keyWord)):

        for items in soup.find_all(class_="mw-content-ltr"):
          if items.find("h3"):
            for header in items.find_all("h3"):
              paragraph = header.find_next("p")
              if bool(re.search(keyWord, paragraph.text)):
                print()
                print("HEADER: " + header.text)
                print(paragraph.text)
                x_string = ""
                for word in paragraph.text:
                  x_string += word
                thread = threading.Thread(target=doDefault, args=(x_string,))
                thread.start()
                while not result_available.wait(timeout=1):
                  pass
                result_available.clear()
                defaultValue = str(lastDigit)
                myValue = input(defaultValue)
                myValue = myValue or defaultValue
                toSend = [int(index), str(myValue)]
                threadSend = threading.Thread(target=acceptResult, args=(toSend, targetColumnName, ))
                threadSend.start()
                while not result_available.wait(timeout=1):
                  pass
                result_available.clear()
                print("DONE")

          else:
            paragraphs = items.find_all("p")
            for p in paragraphs:
              data = p.find_all(text=re.compile(keyWord))
              p_text = p.text
              if data != []:
                for x in data: 
                  print("Part with keyword: ")
                  print(x)
                  print()
                  print("Full paragraph")
                  print(p_text)
                  x_string = ""
                  for word in x:
                    x_string += word
                  thread = threading.Thread(target=doDefault, args=(x_string,))
                  thread.start()
                  while not result_available.wait(timeout=1):
                    pass
                  result_available.clear()
                  defaultValue = str(lastDigit)
                  myValue = input(defaultValue)
                  myValue = myValue or defaultValue
                  toSend = [int(index), str(myValue)]
                  threadSend = threading.Thread(target=acceptResult, args=(toSend, targetColumnName, ))
                  threadSend.start()
                  while not result_available.wait(timeout=1):
                    pass
                  result_available.clear()
                  print("DONE")
      else:
        print("NO RESULT")
    except:
      pass

def main():
    runFor(small_ds)

if __name__ == '__main__':
    main()



************************
row[NAME]  row[Start year]  row[ISO]  row[Num homeless]
DELLA 1949 JPN nan


************************
row[NAME]  row[Start year]  row[ISO]  row[Num homeless]
RENA 1949 PHL nan

TITLE: 1949 Pacific typhoon season - Wikipedia
NO RESULT

************************
row[NAME]  row[Start year]  row[ISO]  row[Num homeless]
JANE 1950 JPN nan

TITLE: Hurricane Jane - Wikipedia
NO RESULT

************************
row[NAME]  row[Start year]  row[ISO]  row[Num homeless]
IRIS 1951 PHL nan

TITLE: 1951 Pacific typhoon season - Wikipedia
NO RESULT

************************
row[NAME]  row[Start year]  row[ISO]  row[Num homeless]
CHARLIE 1951 JAM nan

TITLE: Hurricane Charlie (1951) - Wikipedia
Part with keyword: 
 Charlie caused around $50 million (1951 USD, $380 million 2005 USD) in crop and property damage, 152 deaths, injured 2,000, and left 25,000 homeless.

Full paragraph
The cyclone was compact when it moved through the Lesser Antilles, and the strongest winds on land in 

In [133]:
temp_dataset


,Unnamed: 0,Index,NAME,ISO,Start year,Num affected,Num homeless,Num injured,wikipedia link
0,0,0,DELLA,JPN,1949,194046.0,NaN,NaN,NaN
1,1,1,RENA,PHL,1949,NaN,NaN,NaN,https://en.wikipedia.org/wiki/Typhoon_Rena_(1949)
2,2,2,JANE,JPN,1950,642117.0,NaN,NaN,https://en.wikipedia.org/wiki/Hurricane_Jane
3,3,3,IRIS,PHL,1951,NaN,NaN,NaN,https://en.wikipedia.org/wiki/Typhoon_Iris_(1951)
4,4,4,CHARLIE,JAM,1951,20000.0,25000.0,200.0,https://en.wikipedia.org/wiki/Hurricane_Charli...
5,5,5,RUTH,JPN,1951,NaN,NaN,NaN,https://en.wikipedia.org/wiki/Typhoon_Ruth_(1951)
6,6,6,WANDA,PHL,1951,NaN,NaN,NaN,https://en.wikipedia.org/wiki/Typhoon_Wanda_(1...
7,7,7,AMY,PHL,1951,60000.0,50000.0,NaN,https://en.wikipedia.org/wiki/Typhoon_Amy_(1951)
8,8,8,EMMA,PHL,1952,NaN,NaN,103.0,https://en.wikipedia.org/wiki/Typhoon_Emma_(1952)
9,9,9,TRIX,PHL,1952,NaN,NaN,349.0,https://en.wikipedia.org/wiki/Typhoon_Trix_(1952)
